#### @prompt example

## LangChain with OpenBB
This notebook will leverage the following technologies/frameworks to do some stock research.
It will create an Agent that will fetch information for top performing companies in the top performing industry
It will then leverage Chroma to load the latest earning transcript call for Altria (MO) to allow users to ask few questions
on Altria's latest result.
This work was inspired by this article

- Gemini
- LangChain
- OpenBB (openbb.co)

In [ ]:
!pip install openbb
!pip install langchain
!pip install langchain
!pip install -U langchain-google-genai
!pip install -U -q "google-genai==1.7.0"
!pip install langchain_community
!pip install grandalf
!pip install pypdf
!pip install docx2txt
!pip install chromadb
!pip install wikipedia
!pip install openbb-finviz
!pip install openbb-yfinance

^C
ERROR: Operation cancelled by user
^C
ERROR: Operation cancelled by user


In [ ]:
from openbb import obb
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationTokenBufferMemory
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import create_structured_chat_agent, AgentExecutor




### Creating tools we will need for our stock research

In [ ]:
@tool
def get_industry_performance() -> list:
    """ Return performance by industry for last week, last month, last quarter, last half year and last year"""
    return obb.equity.compare.groups(group='industry', metric='performance').to_llm()

@tool
def get_strong_buy_for_sector(sector : str) -> list :
    """ Return the strong buy recommendation for a given sector"""
    new_sector = '_'.join(sector.lower().split())
    data = obb.equity.screener(provider='finviz', sector=new_sector, recommendation='buy')
    return data.to_llm()

@tool
def get_best_stock_performers_for_sector(sector:str) -> list :
    """ Return the best  10 stock performers for last week and last month for a sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : sector, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=10)
    return data.to_llm()

@tool
def get_best_stock_performers_for_industry(industry:str) -> list :
    """ Return the best  10 stock performers for last week and last month for a sector"""
    data = obb.equity.screener(provider='finviz', filters_dict={'Sector' : industry, 'Performance' : 'Week Up', 'Performance 2' : 'Month Up'}, limit=10)
    return data.to_llm()

@tool
def get_valuation_for_sectors(input:str) -> list:
    """ Return valuation metrics for the sector provided as input"""
    data = obb.equity.compare.groups(group='sector', metric='valuation', provider='finviz').to_df()
    
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )

@tool
def get_valuation_for_industries(input:str) -> list:
    """ Return valuation meetrics for the industry provided as input"""
    data =  obb.equity.compare.groups(group='industry', metric='valuation', provider='finviz').to_df()
    filtered =  data[data.name == input]
    return filtered.to_json(
            orient="records",
            date_format="iso",
            date_unit="s",
        )
@tool
def get_income_statement(ticker:str) -> list:
    """ Return the last 3  income statement for  a given ticker. Limiting the fields returned to avoid max token errors """
    income_stmnts = obb.equity.fundamental.income(symbol=ticker, limit=3, provider='yfinance').to_df()
    return income_stmnts[['period_ending', 'operating_revenue','total_revenue', 'cost_of_revenue', 'gross_profit']]\
                .to_json(orient="records", date_format="iso", date_unit="s")


@tool
def get_valuation_for_company(ticker:str) -> list:
    """ Return Valuation Ratios  for a company"""#
    # requires obb login
    obb.account.login(pat=os.environ['PAT_KEY'])
    return obb.equity.fundamental.ratios_ttm(symbol='AAPL', provider='fmp', limit=1).to_llm()

#get_strong_buy_for_sector.invoke('Consumer Cyclical')
get_valuation_for_sectors.invoke('Financials')

### Chat Memory

In [2]:




MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful stock recommendation assistant , but dont know current events so you should use your tools as much as you can.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

NameError: name 'ChatPromptTemplate' is not defined

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7, max_output_tokens=100)
tools = [get_industry_performance, get_income_statement, get_best_stock_performers_for_industry, get_valuation_for_industries, get_income_statement]
llm_with_tools = llm.bind_tools(tools)


chat_history = []
chat_history.append(HumanMessage(content="Your question here"))
chat_history.append(AIMessage(content="AI response here"))
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=17000,
    memory_key="chat_history",
    return_messages=True
)

# Define your prompt
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="chat_history"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: memory.load_memory_variables(x)["chat_history"],
    }
    | prompt
    | llm_with_tools
    | StrOutputParser()
)


# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)


In [ ]:
input1 = '''
'''
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
print(result['output'])

In [ ]:
input1 = "Now find me valuation metrics for this sector, please use the right function for this"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
print(result['output'])

In [ ]:
!pip install -U langchain-google-genai

In [ ]:
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "Which stock are recommented for this sector best performing sector", "chat_history": chat_history})

In [ ]:
obb.equity.fundamental.income(symbol='AAPL', limit=3, provider='yfinance').to_df()